# SimPy Example for IE 306.02
This example shows how a simple G/G/1 system (1 server with an infinite capacity queue, random interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at randomn times. There is only a single operator in this call center, and picks up the first call waiting when she is available. The customers are assumed to be extremely paint, as they wait as long as it gets to talk to the operator.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt

In [2]:
def format_time(total_seconds):
    secs = total_seconds % 60
    mins = int((total_seconds / 60) % 60)
    hours = int(total_seconds //60 //60)
    return f" {hours:02}:{mins:02}:{secs:04.1f}"
        
Expert_Q_Stat = namedtuple('Expert_Q_Stat' ['enter_e_q', 'exit_e_q'])

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

•The inter-arrival times of calls are exponentially distributed with mean 14.3 min. 

•Incoming calls are first processed by the unskilled front-desk operator who records the personal details of the caller and the nature of the caller's request. When the operator is busy the customers are put on hold (they wait in a FCFS queue). 

•The time ittakes to collect and record the details of a caller is assumed to be LogNormal distributed with mean 7.2and standard deviation of 2.7minutes.

•Once this process is completed, the caller is directed to the expert operator, who tries to help  the  caller  with his/her  problem. As in the previous case, if the expert is busy the customers are put on hold (they wait in a FCFS queue). 

•Every customer that joins the queue leaves the queue without getting service following an independent exponentially distributed time with mean 60 minutes. (reneging).

•The service time of the expert is Exponentially distributed with mean 10.2minutes.

•The  expert  operator  takes  3-min  breaks  randomly  through  out  the  day.  When  the operator decides to take a break, he/she waits until completing all the customers already waiting  for  her/him.  If  new  customers  arrive  during  operators  break,  they  wait  in  the FCFS queue until the operator serves them. The operator resumes service after the break. 

•The  number  of  breaks  the  expert  operator  wishes to  take  during  an  8-hour  shift  is known to be distributed according to a Poisson distribution with a mean of 8 breaks per shift.

•Simulate this system for 1000 and 5000 answered calls separately.

•You should base your code on the SimPypseudocode provided in the jupyter notebook. 

•Collect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

In [3]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3
CALLNUM1=1000
CALLNUM2=5000

LAST_EXIT = 0

#RANDOM_SEED = 971
#random.seed(RANDOM_SEED)

Define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []


break_count = 0



* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.stats = 
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue. \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue. \t at {format_time(self.finish_operator)}")
        
        if self.env.now >=60*60*8:
            #Shop is closed
            print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
            print(f"-- shop is closed")
        else:
            with expert.request() as exp_req: 
                enter_e_q = self.env.now
                self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
                results = yield exp_req | env.timeout(self.patience)
                self.wait_exp=self.env.now-self.finish_operator
                second_queue_w_times.append(self.wait_exp) 
                exit_e_q = self.env.now
                global expert_queue_log
                expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

                if exp_req not in results:
                    #Leave queue, regened
                    print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
                    print(f"-- after waiting {format_time(self.wait_exp)}")

                else:
                # We got to the expert
                    print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                    if self.wait_exp:
                        print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                    yield self.env.process(self.ask_question_expert())
                    self.finish_expert=self.env.now
                    print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        # print("%s ask_question-op at  %g , service_opr= %g" % (self.id, self.env.now, service_opr))
        yield self.env.timeout(service_opr)
        # print("%s ask_question-op at  %g finished " % (self.id, self.env.now))

        first_service_times.append(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        # print("%s ask_question-exp at  %g , service_expr= %g" % (self.id, self.env.now,service_exp))
        yield self.env.timeout(service_exp)
        second_service_times.append(service_exp)   # Why is this after yield

In [7]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        if env.now >= 60*60*8:
            break
        time_between_next_customer = random.expovariate(1.0 / INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)
        if env.now + time_between_next_customer >= 60*60*8:
            break
        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while True:
        time_to_break = random.expovariate(1/60/60)
        if env.now + time_to_break >= 60*60*8:
            break
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1


In [8]:
env = simpy.Environment()
CUSTOMER_NUM=CALLNUM2
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00.0
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00.0
Cust    1 entered the 		 EXPERT Queue. 	 at  00:00:06.1
Cust    1 started talking with 	 EXPERT 	 at  00:00:06.1
Cust    1 exited 		 NORMALLY 	 at  00:00:06.3
Cust    2 entered the 		 FRONT Queue. 	 at  00:00:39.1
Cust    2 started talking with 	 FRONT Desk 	 at  00:00:39.1
Cust    3 entered the 		 FRONT Queue. 	 at  00:00:43.0
Cust    4 entered the 		 FRONT Queue. 	 at  00:00:44.7
Cust    2 entered the 		 EXPERT Queue. 	 at  00:00:47.6
Cust    3 started talking with 	 FRONT Desk 	 at  00:00:47.6
Cust    3 waited in 		 FRONT Queue 	 for 00:00:04.7
Cust    2 started talking with 	 EXPERT 	 at  00:00:47.6
Cust    3 entered the 		 EXPERT Queue. 	 at  00:00:56.3
Cust    4 started talking with 	 FRONT Desk 	 at  00:00:56.3
Cust    4 waited in 		 FRONT Queue 	 for 00:00:11.6
Cust    5 entered the 		 FRONT Queue. 	 at  00:00:58.0
Cust    4 entered the 		 EXPERT Queue. 	 at  00:0

Cust  213 entered the 		 FRONT Queue. 	 at  00:52:23.5
Cust  213 started talking with 	 FRONT Desk 	 at  00:52:23.5
Cust  213 entered the 		 EXPERT Queue. 	 at  00:52:30.9
Cust  213 started talking with 	 EXPERT 	 at  00:52:30.9
Cust  213 exited 		 NORMALLY 	 at  00:52:34.6
Cust  214 entered the 		 FRONT Queue. 	 at  00:52:54.8
Cust  214 started talking with 	 FRONT Desk 	 at  00:52:54.8
Cust  215 entered the 		 FRONT Queue. 	 at  00:52:56.4
Cust  214 entered the 		 EXPERT Queue. 	 at  00:53:01.3
Cust  215 started talking with 	 FRONT Desk 	 at  00:53:01.3
Cust  215 waited in 		 FRONT Queue 	 for 00:00:04.9
Cust  214 started talking with 	 EXPERT 	 at  00:53:01.3
Cust  216 entered the 		 FRONT Queue. 	 at  00:53:07.0
Cust  215 entered the 		 EXPERT Queue. 	 at  00:53:09.3
Cust  216 started talking with 	 FRONT Desk 	 at  00:53:09.3
Cust  216 waited in 		 FRONT Queue 	 for 00:00:02.4
Cust  214 exited 		 NORMALLY 	 at  00:53:09.6
Cust  215 started talking with 	 EXPERT 	 at  00:53:09.6
C

Cust  440 started talking with 	 FRONT Desk 	 at  01:55:13.1
Cust  440 entered the 		 EXPERT Queue. 	 at  01:55:17.2
Cust  441 entered the 		 FRONT Queue. 	 at  01:55:20.2
Cust  441 started talking with 	 FRONT Desk 	 at  01:55:20.2
Cust  440 exited 		 SUDDENLY 	 at  01:55:21.6
-- after waiting  00:00:04.4
Cust  439 exited 		 NORMALLY 	 at  01:55:23.0
Cust  442 entered the 		 FRONT Queue. 	 at  01:55:23.2
Cust  441 entered the 		 EXPERT Queue. 	 at  01:55:27.1
Cust  442 started talking with 	 FRONT Desk 	 at  01:55:27.1
Cust  442 waited in 		 FRONT Queue 	 for 00:00:03.9
Cust  441 started talking with 	 EXPERT 	 at  01:55:27.1
Cust  442 entered the 		 EXPERT Queue. 	 at  01:55:33.1
Cust  442 exited 		 SUDDENLY 	 at  01:55:33.6
-- after waiting  00:00:00.5
Cust  443 entered the 		 FRONT Queue. 	 at  01:55:38.3
Cust  443 started talking with 	 FRONT Desk 	 at  01:55:38.3
Cust  443 entered the 		 EXPERT Queue. 	 at  01:55:44.1
Cust  441 exited 		 NORMALLY 	 at  01:55:47.7
Cust  443 starte

Cust  631 started talking with 	 FRONT Desk 	 at  02:38:43.2
Cust  630 exited 		 NORMALLY 	 at  02:38:45.0
Cust  631 entered the 		 EXPERT Queue. 	 at  02:38:48.7
Cust  631 started talking with 	 EXPERT 	 at  02:38:48.7
Cust  631 exited 		 NORMALLY 	 at  02:38:56.9
Cust  632 entered the 		 FRONT Queue. 	 at  02:39:16.4
Cust  632 started talking with 	 FRONT Desk 	 at  02:39:16.4
Cust  633 entered the 		 FRONT Queue. 	 at  02:39:25.6
Cust  632 entered the 		 EXPERT Queue. 	 at  02:39:26.6
Cust  633 started talking with 	 FRONT Desk 	 at  02:39:26.6
Cust  633 waited in 		 FRONT Queue 	 for 00:00:00.9
Cust  632 started talking with 	 EXPERT 	 at  02:39:26.6
Cust  634 entered the 		 FRONT Queue. 	 at  02:39:27.0
Cust  632 exited 		 NORMALLY 	 at  02:39:27.7
Cust  633 entered the 		 EXPERT Queue. 	 at  02:39:38.6
Cust  634 started talking with 	 FRONT Desk 	 at  02:39:38.6
Cust  634 waited in 		 FRONT Queue 	 for 00:00:11.6
Cust  633 started talking with 	 EXPERT 	 at  02:39:38.6
Cust  635 

Cust  823 started talking with 	 EXPERT 	 at  03:26:11.5
Cust  823 waited in 		 EXPERT Queue 	 for 00:00:02.6
Cust  823 exited 		 NORMALLY 	 at  03:26:56.5
Cust  824 entered the 		 FRONT Queue. 	 at  03:26:57.4
Cust  824 started talking with 	 FRONT Desk 	 at  03:26:57.4
Cust  824 entered the 		 EXPERT Queue. 	 at  03:27:02.0
Cust  824 started talking with 	 EXPERT 	 at  03:27:02.0
Cust  824 exited 		 NORMALLY 	 at  03:27:04.2
Cust  825 entered the 		 FRONT Queue. 	 at  03:27:32.6
Cust  825 started talking with 	 FRONT Desk 	 at  03:27:32.6
Cust  826 entered the 		 FRONT Queue. 	 at  03:27:34.6
Cust  825 entered the 		 EXPERT Queue. 	 at  03:27:40.3
Cust  826 started talking with 	 FRONT Desk 	 at  03:27:40.3
Cust  826 waited in 		 FRONT Queue 	 for 00:00:05.7
Cust  825 started talking with 	 EXPERT 	 at  03:27:40.3
Cust  827 entered the 		 FRONT Queue. 	 at  03:27:41.6
Cust  828 entered the 		 FRONT Queue. 	 at  03:27:45.9
Cust  826 entered the 		 EXPERT Queue. 	 at  03:27:46.7
Cust  

Cust 1008 started talking with 	 EXPERT 	 at  04:08:00.3
Cust 1008 exited 		 NORMALLY 	 at  04:08:02.1
Cust 1009 entered the 		 FRONT Queue. 	 at  04:08:16.8
Cust 1009 started talking with 	 FRONT Desk 	 at  04:08:16.8
Cust 1009 entered the 		 EXPERT Queue. 	 at  04:08:23.7
Cust 1009 started talking with 	 EXPERT 	 at  04:08:23.7
Cust 1009 exited 		 NORMALLY 	 at  04:08:25.5
Cust 1010 entered the 		 FRONT Queue. 	 at  04:08:35.5
Cust 1010 started talking with 	 FRONT Desk 	 at  04:08:35.5
Cust 1011 entered the 		 FRONT Queue. 	 at  04:08:40.1
Cust 1010 entered the 		 EXPERT Queue. 	 at  04:08:41.7
Cust 1011 started talking with 	 FRONT Desk 	 at  04:08:41.7
Cust 1011 waited in 		 FRONT Queue 	 for 00:00:01.7
Cust 1010 started talking with 	 EXPERT 	 at  04:08:41.7
Cust 1012 entered the 		 FRONT Queue. 	 at  04:08:42.3
Cust 1011 entered the 		 EXPERT Queue. 	 at  04:08:48.6
Cust 1012 started talking with 	 FRONT Desk 	 at  04:08:48.6
Cust 1012 waited in 		 FRONT Queue 	 for 00:00:06.2
C

Cust 1205 entered the 		 FRONT Queue. 	 at  04:56:49.7
Cust 1204 entered the 		 EXPERT Queue. 	 at  04:56:54.2
Cust 1205 started talking with 	 FRONT Desk 	 at  04:56:54.2
Cust 1205 waited in 		 FRONT Queue 	 for 00:00:04.5
Cust 1204 started talking with 	 EXPERT 	 at  04:56:54.2
Cust 1204 exited 		 NORMALLY 	 at  04:56:58.3
Cust 1206 entered the 		 FRONT Queue. 	 at  04:57:00.8
Cust 1205 entered the 		 EXPERT Queue. 	 at  04:57:02.3
Cust 1206 started talking with 	 FRONT Desk 	 at  04:57:02.3
Cust 1206 waited in 		 FRONT Queue 	 for 00:00:01.4
Cust 1205 started talking with 	 EXPERT 	 at  04:57:02.3
Cust 1205 exited 		 NORMALLY 	 at  04:57:03.0
Cust 1207 entered the 		 FRONT Queue. 	 at  04:57:05.4
Cust 1206 entered the 		 EXPERT Queue. 	 at  04:57:09.6
Cust 1207 started talking with 	 FRONT Desk 	 at  04:57:09.6
Cust 1207 waited in 		 FRONT Queue 	 for 00:00:04.2
Cust 1206 started talking with 	 EXPERT 	 at  04:57:09.6
Cust 1208 entered the 		 FRONT Queue. 	 at  04:57:10.9
Cust 1209 

Cust 1397 entered the 		 EXPERT Queue. 	 at  05:46:30.9
Cust 1398 started talking with 	 FRONT Desk 	 at  05:46:30.9
Cust 1398 waited in 		 FRONT Queue 	 for 00:00:05.4
Cust 1398 entered the 		 EXPERT Queue. 	 at  05:46:36.8
Cust 1399 entered the 		 FRONT Queue. 	 at  05:46:39.6
Cust 1399 started talking with 	 FRONT Desk 	 at  05:46:39.6
Cust 1395 exited 		 NORMALLY 	 at  05:46:43.9
Cust 1396 started talking with 	 EXPERT 	 at  05:46:43.9
Cust 1396 waited in 		 EXPERT Queue 	 for 00:00:25.8
Cust 1396 exited 		 NORMALLY 	 at  05:46:46.8
Cust 1397 started talking with 	 EXPERT 	 at  05:46:46.8
Cust 1397 waited in 		 EXPERT Queue 	 for 00:00:15.9
Cust 1400 entered the 		 FRONT Queue. 	 at  05:46:47.6
Cust 1399 entered the 		 EXPERT Queue. 	 at  05:46:47.9
Cust 1400 started talking with 	 FRONT Desk 	 at  05:46:47.9
Cust 1400 waited in 		 FRONT Queue 	 for 00:00:00.3
Cust 1401 entered the 		 FRONT Queue. 	 at  05:46:55.5
Cust 1400 entered the 		 EXPERT Queue. 	 at  05:46:55.7
Cust 1401 st

Cust 1622 entered the 		 FRONT Queue. 	 at  06:39:27.2
Cust 1622 started talking with 	 FRONT Desk 	 at  06:39:27.2
Cust 1622 entered the 		 EXPERT Queue. 	 at  06:39:35.6
Cust 1622 started talking with 	 EXPERT 	 at  06:39:35.6
Cust 1623 entered the 		 FRONT Queue. 	 at  06:39:38.5
Cust 1623 started talking with 	 FRONT Desk 	 at  06:39:38.5
Cust 1623 entered the 		 EXPERT Queue. 	 at  06:39:45.7
Cust 1622 exited 		 NORMALLY 	 at  06:39:50.0
Cust 1623 started talking with 	 EXPERT 	 at  06:39:50.0
Cust 1623 waited in 		 EXPERT Queue 	 for 00:00:04.2
Cust 1624 entered the 		 FRONT Queue. 	 at  06:39:51.3
Cust 1624 started talking with 	 FRONT Desk 	 at  06:39:51.3
Cust 1625 entered the 		 FRONT Queue. 	 at  06:39:55.6
Cust 1623 exited 		 NORMALLY 	 at  06:39:55.6
Cust 1624 entered the 		 EXPERT Queue. 	 at  06:39:56.3
Cust 1625 started talking with 	 FRONT Desk 	 at  06:39:56.3
Cust 1625 waited in 		 FRONT Queue 	 for 00:00:00.7
Cust 1624 started talking with 	 EXPERT 	 at  06:39:56.3


Cust 1871 started talking with 	 EXPERT 	 at  07:37:18.5
Cust 1871 waited in 		 EXPERT Queue 	 for 00:00:03.8
Cust 1872 entered the 		 FRONT Queue. 	 at  07:37:42.7
Cust 1872 started talking with 	 FRONT Desk 	 at  07:37:42.7
Cust 1872 entered the 		 EXPERT Queue. 	 at  07:37:50.0
Cust 1871 exited 		 NORMALLY 	 at  07:37:56.7
Cust 1872 started talking with 	 EXPERT 	 at  07:37:56.7
Cust 1872 waited in 		 EXPERT Queue 	 for 00:00:06.7
Cust 1872 exited 		 NORMALLY 	 at  07:38:15.8
Cust 1873 entered the 		 FRONT Queue. 	 at  07:38:33.1
Cust 1873 started talking with 	 FRONT Desk 	 at  07:38:33.1
Cust 1873 entered the 		 EXPERT Queue. 	 at  07:38:38.0
Cust 1873 started talking with 	 EXPERT 	 at  07:38:38.0
Cust 1874 entered the 		 FRONT Queue. 	 at  07:38:44.9
Cust 1874 started talking with 	 FRONT Desk 	 at  07:38:44.9
Cust 1873 exited 		 NORMALLY 	 at  07:38:48.5
Cust 1874 entered the 		 EXPERT Queue. 	 at  07:38:51.5
Cust 1874 started talking with 	 EXPERT 	 at  07:38:51.5
Cust 1874 ex

In [9]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0)]
    


print(expert_queue_log)
stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits)
}
print(stats)

13


In [10]:
print (len(first_queue_w_times))

1955


In [11]:
print (len(second_queue_w_times))

1955


In [12]:
print(format_time(env.now))
print(format_time(LAST_EXIT))

 08:02:45.8
 08:00:05.2
